<a href="https://colab.research.google.com/github/azarshka/Linear-Regression-sample/blob/main/Logistic_Regression_PySpark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Logistic Regression PySpark ML

In [1]:
!pip3 install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression

# Initialize SparkSession
spark = SparkSession.builder \
                    .appName("LogisticRegressionExample") \
                    .master("local[*]") \
                    .config("spark.executor.memory", "4g") \
                    .config("spark.driver.memory", "2g") \
                    .config("spark.executor.cores", "2") \
                    .config("spark.sql.inMemoryColumnarStorage.compressed", "true") \
                    .getOrCreate()

spark

In [3]:
import gzip

f = gzip.open('kddcup.data_10_percent.gz')
f.close()

In [4]:
df = spark.read.csv("/content/kddcup.data_10_percent.gz", header = False)

# Display the DataFrame
df.show(5)

# more info
print(df.count())
print(df.rdd.getNumPartitions())

+---+---+----+---+---+----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|_c0|_c1| _c2|_c3|_c4| _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|   _c41|
+---+---+----+---+---+----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|  0|tcp|http| SF|181|5450|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   8|   8|0.00|0.00|0.00|0.00|1.00|0.00|0.00|   9|   9|1.00|0.00|0.11|0.00|0.00|0.00|0.00|0.00|normal.|
|  0|tcp|http| SF|239| 486|  0|  0|  0|  0|   0|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   8|   8|0.00|0.00|0.00|0.00|1.00|0.00|0.00|  19|  19|1.00|0.00|0.05

In [5]:
# for the 10% of data:
df = df.repartition(2)
df.rdd.getNumPartitions()

2


Add header

In [6]:


df = df.withColumnRenamed("_c0","duration") \
      .withColumnRenamed("_c1","protocol_type")\
      .withColumnRenamed("_c2","service")\
      .withColumnRenamed("_c3","flag")\
      .withColumnRenamed("_c4","src_bytes")\
      .withColumnRenamed("_c5","dst_bytes")\
      .withColumnRenamed("_c6","land")\
      .withColumnRenamed("_c7","wrong_fragment")\
      .withColumnRenamed("_c8","urgent")\
      .withColumnRenamed("_c9","host")\
      .withColumnRenamed("_c10","num_failed_logins")\
      .withColumnRenamed("_c11","logged_in")\
      .withColumnRenamed("_c12","num_compromised")\
      .withColumnRenamed("_c13","root_shell")\
      .withColumnRenamed("_c14","su_attempted")\
      .withColumnRenamed("_c15","num_root")\
      .withColumnRenamed("_c16","num_file_creations")\
      .withColumnRenamed("_c17","num_shells")\
      .withColumnRenamed("_c18","num_access_files")\
      .withColumnRenamed("_c19","num_outbound_cmds")\
      .withColumnRenamed("_c20","is_host_login")\
      .withColumnRenamed("_c21","is_guest_login")\
      .withColumnRenamed("_c22","count")\
      .withColumnRenamed("_c23","srv_count")\
      .withColumnRenamed("_c24","serror_rate")\
      .withColumnRenamed("_c25","srv_serror_rate")\
      .withColumnRenamed("_c26","rerror_rate")\
      .withColumnRenamed("_c27","srv_rerror_rate")\
      .withColumnRenamed("_c28","same_srv_rate")\
      .withColumnRenamed("_c29","diff_srv_rate")\
      .withColumnRenamed("_c30","srv_diff_host_rate")\
      .withColumnRenamed("_c31","dst_host_count")\
      .withColumnRenamed("_c32","dst_host_srv_count")\
      .withColumnRenamed("_c33","dst_host_same_srv_rate")\
      .withColumnRenamed("_c34","dst_host_diff_srv_rate")\
      .withColumnRenamed("_c35","dst_host_same_src_port_rate")\
      .withColumnRenamed("_c36","dst_host_srv_diff_host_rate")\
      .withColumnRenamed("_c37","dst_host_serror_rate")\
      .withColumnRenamed("_c38","dst_host_srv_serror_rate")\
      .withColumnRenamed("_c39","dst_host_rerror_rate")\
      .withColumnRenamed("_c40","dst_host_srv_rerror_rate")\
      .withColumnRenamed("_c41","connection_status")

df.show(5)

+--------+-------------+--------+----+---------+---------+----+--------------+------+----+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-----------------+
|duration|protocol_type| service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|host|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_hos

Check the Binary labels

In [7]:
df.select("connection_status").distinct().show(15)

+-----------------+
|connection_status|
+-----------------+
|     warezmaster.|
|           smurf.|
|             pod.|
|            nmap.|
|            imap.|
|    guess_passwd.|
|         ipsweep.|
|       portsweep.|
|           satan.|
|            land.|
|      loadmodule.|
|       ftp_write.|
| buffer_overflow.|
|         rootkit.|
|     warezclient.|
+-----------------+
only showing top 15 rows



In [8]:
from pyspark.sql.functions import when

df = df.withColumn("label", when(df["connection_status"] != 'normal.', 1).otherwise(0))

df.select("label").distinct().show()

+-----+
|label|
+-----+
|    1|
|    0|
+-----+



Count the labels

In [9]:
df.groupBy("label").count().show()

# ~20% normal traffic
# ~80% attack traffic


# dealing with imbalanced labels:
 # Resampling
 # Wieghted Loss
 # Data Augmentation: SMOTE method
 # Ensemble methods
 # Evaluation metrics: precision, recall, F1 socre, AUC-ROC

+-----+------+
|label| count|
+-----+------+
|    1|396743|
|    0| 97278|
+-----+------+




StringIndexer

In [10]:
from pyspark.ml.feature import StringIndexer

columns_to_index = ["protocol_type", "service", "flag"]

for column in columns_to_index:
  indexer = StringIndexer(inputCol = column, outputCol = column + "_indexed")
  df = indexer.fit(df).transform(df)

df.show(5)

+--------+-------------+--------+----+---------+---------+----+--------------+------+----+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-----------------+-----+---------------------+---------------+------------+
|duration|protocol_type| service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|host|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate

Missing values

In [11]:
from pyspark.sql.functions import col, sum

missing_values = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

missing_values.show()

+--------+-------------+-------+----+---------+---------+----+--------------+------+----+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-----------------+-----+---------------------+---------------+------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|host|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|s

To assess the linear separability of a feature

In [12]:
from pyspark.sql.functions import avg, stddev

# avg & std for class 0
class_0_stats = df.filter(df['label'] == 0).select(avg('src_bytes').alias("avg_src_bytes_0"),
                                                   stddev('src_bytes').alias("stddev_src_bytes_0")).first()

# avg & std for class 1
class_1_stats = df.filter(df['label'] == 1).select(avg('src_bytes').alias("avg_src_bytes_1"),
                                                   stddev('src_bytes').alias("stddev_src_bytes_1")).first()


print("Class 0 distribution: ")
print("AVG src_bytes: ", class_0_stats["avg_src_bytes_0"])
print("STD src_bytes: ", class_0_stats["stddev_src_bytes_0"])

print("Class 1 distribution: ")
print("AVG src_bytes: ", class_1_stats["avg_src_bytes_1"])
print("STD src_bytes: ", class_1_stats["stddev_src_bytes_1"])

Class 0 distribution: 
AVG src_bytes:  1157.047523592179
STD src_bytes:  34226.1247180511
Class 1 distribution: 
AVG src_bytes:  3483.7659517622237
STD src_bytes:  1102603.8255053805


VectorAssembler

In [13]:
df.printSchema()

root
 |-- duration: string (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: string (nullable = true)
 |-- dst_bytes: string (nullable = true)
 |-- land: string (nullable = true)
 |-- wrong_fragment: string (nullable = true)
 |-- urgent: string (nullable = true)
 |-- host: string (nullable = true)
 |-- num_failed_logins: string (nullable = true)
 |-- logged_in: string (nullable = true)
 |-- num_compromised: string (nullable = true)
 |-- root_shell: string (nullable = true)
 |-- su_attempted: string (nullable = true)
 |-- num_root: string (nullable = true)
 |-- num_file_creations: string (nullable = true)
 |-- num_shells: string (nullable = true)
 |-- num_access_files: string (nullable = true)
 |-- num_outbound_cmds: string (nullable = true)
 |-- is_host_login: string (nullable = true)
 |-- is_guest_login: string (nullable = true)
 |-- count: string (nullable = true)
 |-- srv_count: s

In [14]:
from pyspark.sql.functions import col

for column in df.columns:
  df = df.withColumn(column, col(column).cast("double"))

In [15]:
df.printSchema()

root
 |-- duration: double (nullable = true)
 |-- protocol_type: double (nullable = true)
 |-- service: double (nullable = true)
 |-- flag: double (nullable = true)
 |-- src_bytes: double (nullable = true)
 |-- dst_bytes: double (nullable = true)
 |-- land: double (nullable = true)
 |-- wrong_fragment: double (nullable = true)
 |-- urgent: double (nullable = true)
 |-- host: double (nullable = true)
 |-- num_failed_logins: double (nullable = true)
 |-- logged_in: double (nullable = true)
 |-- num_compromised: double (nullable = true)
 |-- root_shell: double (nullable = true)
 |-- su_attempted: double (nullable = true)
 |-- num_root: double (nullable = true)
 |-- num_file_creations: double (nullable = true)
 |-- num_shells: double (nullable = true)
 |-- num_access_files: double (nullable = true)
 |-- num_outbound_cmds: double (nullable = true)
 |-- is_host_login: double (nullable = true)
 |-- is_guest_login: double (nullable = true)
 |-- count: double (nullable = true)
 |-- srv_count: d

In [16]:
assembler = VectorAssembler(inputCols = ["src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
                                         "host", "num_failed_logins", "logged_in", "num_compromised",
                                         "root_shell", "num_root", "num_file_creations", "num_shells",
                                         "num_outbound_cmds", "is_host_login", "count", "srv_count",
                                         "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
                                         "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
                                         "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate",
                                         "dst_host_srv_rerror_rate",
                                         "protocol_type_indexed", "service_indexed", "flag_indexed"],
                            outputCol = "features")

data = assembler.transform(df)

data = data.select('features', 'label')
data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(31,[15,16,17,18,...|  1.0|
|(31,[0,1,15,16,21...|  0.0|
|(31,[0,7,15,16,21...|  0.0|
|(31,[15,16,17,18,...|  1.0|
|(31,[0,1,7,15,16,...|  0.0|
+--------------------+-----+
only showing top 5 rows



In [17]:
data.show(5, truncate = False)

#Sparse Vectors

+-----------------------------------------------------------------------------------------------------+-----+
|features                                                                                             |label|
+-----------------------------------------------------------------------------------------------------+-----+
|(31,[15,16,17,18,21,22,24,25,26,28,29,30],[110.0,10.0,1.0,1.0,0.09,0.06,255.0,15.0,0.06,1.0,1.0,1.0])|1.0  |
|(31,[0,1,15,16,21,23,24,25,26,28,29],[43.0,43.0,6.0,67.0,1.0,0.06,255.0,255.0,1.0,2.0,5.0])          |0.0  |
|(31,[0,7,15,16,21,24,25,26,28,29],[12324.0,1.0,1.0,1.0,1.0,144.0,45.0,0.31,1.0,6.0])                 |0.0  |
|(31,[15,16,17,18,21,22,24,25,26,28,29,30],[130.0,11.0,1.0,1.0,0.08,0.05,255.0,8.0,0.03,1.0,1.0,1.0]) |1.0  |
|(31,[0,1,7,15,16,21,24,25,26,28,29],[304.0,357.0,1.0,25.0,25.0,1.0,103.0,255.0,1.0,1.0,2.0])         |0.0  |
+-----------------------------------------------------------------------------------------------------+-----+
only showi

In [18]:
# Extract the values from Sparse Vectors to the list format
selected_data = data.select('features').limit(2).collect()

for row in selected_data:
  dense_vector = row[0].toArray()
  print(dense_vector)

[  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     1.     1.     0.     0.     1.
   1.     1.     0.     0.    21.   255.     1.     0.39   1.     2.
   2.  ]
[832.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.
   0.   1.   1.   0.   0.   0.   0.   1.   0.   0.   3.  30.   1.   0.
   1.   6.   0.]



StandardScaler

In [19]:
scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')

scaler_model = scaler.fit(data)
data = scaler_model.transform(data)

data = data.select("scaledFeatures", "label")
data.show(3, truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|scaledFeatures                                                                                                                                                                                                                                                                    |label|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(31,[15,16,17,18,21,22,24,25,26,28,29,30],[0.5160747620652872,0.04059713227290855,2.6266232258670694,2.6245576721666946,0.23184553320898044,0.72987823

In [21]:
#Split data
train_data, test_data = data.randomSplit([0.7, 0.3], seed = 1234)

Apply Logistic Regression model

In [22]:
lr = LogisticRegression(featuresCol = "scaledFeatures", labelCol = 'label',
                        threshold = 0.5, regParam = 0.01)

lr_model = lr.fit(train_data)

lr_predictions_train = lr_model.transform(train_data)
lr_predictions_test = lr_model.transform(test_data)

In [23]:
lr_predictions_test.select("label", "prediction").show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 10 rows



**Evaluate the model**


Confusion Matrix

In [24]:
confusion_matrix = lr_predictions_test.groupBy("label", "prediction").count()
confusion_matrix.show()

+-----+----------+------+
|label|prediction| count|
+-----+----------+------+
|  1.0|       1.0|116916|
|  0.0|       1.0|   458|
|  1.0|       0.0|  1324|
|  0.0|       0.0| 28905|
+-----+----------+------+



In [25]:
cm_pandas = confusion_matrix.toPandas()
cm_pandas.pivot(index = 'label', columns = 'prediction', values = 'count')

prediction,0.0,1.0
label,,
0.0,28905,458
1.0,1324,116916



Additional Performance metrics

In [26]:
tp = lr_predictions_test[(lr_predictions_test.label == 1) & (lr_predictions_test.prediction == 1)].count()
fp = lr_predictions_test[(lr_predictions_test.label == 0) & (lr_predictions_test.prediction == 1)].count()
fn = lr_predictions_test[(lr_predictions_test.label == 1) & (lr_predictions_test.prediction == 0)].count()
tn = lr_predictions_test[(lr_predictions_test.label == 0) & (lr_predictions_test.prediction == 0)].count()

In [27]:
accuracy = (tp + tn) / (tp + fp + fn + tn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision * recall) / (precision + recall)

print('accuracy: ', round(accuracy, 4) * 100)
print('precision: ', round(precision, 4) * 100)
print('recall: ', round(recall, 4) * 100)
print('f1 score: ', round(f1, 4) * 100)

accuracy:  98.79
precision:  99.61
recall:  98.88
f1 score:  99.24


In [33]:
!jupyter nbconvert /content/Logistic_Regression_PySpark_ML.ipynb --to html

[NbConvertApp] Converting notebook /content/Logistic_Regression_PySpark_ML.ipynb to html
[NbConvertApp] Writing 367388 bytes to /content/Logistic_Regression_PySpark_ML.html
